# Information Extraction - Assignment
This assignment is based on the Information Extraction lecture and the lab.

Name : Sai Krishna Lakshminarayanan

Student ID: 18230229

In [390]:
#Replicating the same code as providied in the assignment question
import nltk
import re
from statistics import mode

In [391]:
#Replicating the same code as providied in the assignment question
inputfile='football_players.txt' #Location of the file
buf=open(inputfile, encoding="UTF-8")
list_of_doc=buf.read().split('\n')

# Task 1 (10 Marks)
Write a function that takes each document and performs:
1) sentence segmentation 2) tokenization 3) part-of-speech tagging

Please keep in mind that the expected output is a list within a list as shown below.


In [392]:
line = []
for i in list_of_doc:
    if len(i) != 0:#removing the empty lines present in the given document
        line.append(i)
list_of_doc = line
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
def ie_preprocess(document):#to preprocess the document
    #code goes here
    sentence = sent_tokenize(document)# Converting string into list of sentences present in the document
    pos_sentences = []
    for i in sentence:
        text = word_tokenize(i) # Tokenizing every sentence in the document
        pos_sentences.append(nltk.pos_tag(text)) # Tagging of the tokenizeed sentence
    return pos_sentences

Run the following code to check your result for the first document (Ronaldo).

In [393]:
#checking the above code
first_doc=list_of_doc[0]
pos_sent=ie_preprocess(first_doc)
pos_sent[1]

[('He', 'PRP'),
 ('is', 'VBZ'),
 ('a', 'DT'),
 ('forward', 'NN'),
 ('and', 'CC'),
 ('serves', 'NNS'),
 ('as', 'IN'),
 ('captain', 'NN'),
 ('for', 'IN'),
 ('Portugal', 'NNP'),
 ('.', '.')]

Expected output
 [...[('He', 'PRP'),
  ('is', 'VBZ'),
  ('a', 'DT'),
  ('forward', 'NN'),
  ('and', 'CC'),
  ('serves', 'NNS'),
  ('as', 'IN'),
  ('captain', 'NN'),
  ('for', 'IN'),
  ('Portugal', 'NNP'),
  ('.', '.')], ...]

# Task 2 (20 Marks)
Write a function that will take the list of tokens with POS tags for each sentence and returns the named entities (NE). 

Hint: Use binary=True while calling NE chunk function

In [394]:
#This function will be called later in task 3, 4 also for searching Organisation and others
#Therefore two extra parameters x and y are adding inorder to ensure this function can be used there too
#By default x is true and y is "NE" in order to fulfill the task 2 directly without specifying x and y
def named_entity_finding(pos_sent, x=True, y="NE"):#to find the named entities in the document
    named_entities=[]
    b = nltk.ne_chunk(pos_sent, binary=x)#In this binary is equal to the value given by user. By default, binary=True
    for subtree in b.subtrees():
        if subtree.label() == y:#when the value is equal to the given condition. By default, y = "NE"
            a = ""
            for leaf in subtree.leaves():
                a = a + leaf[0] + " "
            named_entities.append(a.strip())#Adding all the named entities
    return named_entities
#Cross checking the above code
pos_sents=ie_preprocess(list_of_doc[0])
named_entity_finding(pos_sents[0])

['Cristiano Ronaldo',
 'Santos Aveiro',
 'ComM',
 'GOIH',
 'Portuguese',
 'Spanish',
 'Real Madrid',
 'Portugal']

Expected output ['Cristiano Ronaldo',
 'Santos Aveiro',
 'ComM',
 'GOIH',
 'Portuguese',
 'Portuguese',
 'Spanish',
 'Real Madrid',
 'Portugal']

# Task 3 (10 Marks)

Now use the named_entity_finding() function to extract all NEs for each document.

Hint: pos_sents holds the list of lists of tokens with POS tags

In [395]:
import itertools
def NE_flat_list_fn(pos_sents): 
    NE=[]
    for i in pos_sents:
            pos_sents = ie_preprocess(i)#tagging every sentence in the document using the for loop
            for pos_sent in pos_sents:
                a = named_entity_finding(pos_sent)#Finding named entities for each sentence in the document
                if len(a) != 0:
                    NE.append(a)#storing it in NE[] by appending the values of a
    #Single line code here. Flatten the list of lists to the single list NE_flat_list
    NE_flat_list = list(itertools.chain.from_iterable(NE))
    return NE_flat_list
ne_flat = NE_flat_list_fn(list_of_doc)
list(set(ne_flat))[4:7]# Taking the unqiue named entities by using set function and showing them as a list 

['Luis Suárez', 'PSG', 'Germany']

# Task 4 (40 Marks)

Write functions to extract the name of the player, country of origin and date of birth as well as the following relations: team(s) of the player and position(s) of the player.

Hint: Use the re.compile() function to create the extraction patterns

Reference: https://docs.python.org/3/howto/regex.html

In [396]:
def name_of_the_player(doc):#to find the name of the player
    t = []
    name = []

    pos_sents = ie_preprocess(doc)
    tree = nltk.ne_chunk(pos_sents[0])

    for subtree in tree.subtrees():
        if subtree.label() == "PERSON":#when the value is equal to the PERSON entity
            a = ""#storing the names in a
            for leaf in subtree.leaves():
                a = a + leaf[0] + " "
            name.append(a.strip())# appending the values of a into name
    name=" ".join(name)
    return name # joining the segment of the name into complete one




name_of_the_player(list_of_doc[6])


'David Robert Joseph Beckham'

In [397]:
from difflib import SequenceMatcher
def country_of_origin(doc):#to obtain the country of origin of the player
    sent = sent_tokenize(doc)[0]
    a = []
    e = []
      
    b = re.search(r'((?:\w+\W+){,3})(footballer)', sent)#searcing using regex to find the nationality by preceding word of footballer
    s = "".join(list(b.groups()))
    c = re.search(r'((\w+ ){1})professional', s)#searcing using regex to find the nationality by preceding word of professional
    c = c.groups()[0]
    tag = ie_preprocess(s)[0]#tagging of the sentences
    tree = nltk.ne_chunk(tag)
    born = []

    for subtree in tree.subtrees():
            if subtree.label() == 'GPE':#checking for the GPE tagging entities in the sentences
                entity = ""
                for leaf in subtree.leaves():
                    entity = entity + leaf[0] + " "
                born.append(entity.strip())#appending the values to born

    if len(born) == 0:
        if len(c) != 0:#when nltk can't find the country before, now using ways of searching it by finding it near player in the document
            born = nltk.word_tokenize(c) 
    d = ['Spain', 'England', 'Portugal', 'Wales', 'England', 'Brazil', 'Germany', 'Argentina', 'Sweden']#countries present in document
    for x in d:
        e.append(SequenceMatcher(None, born[0], x).ratio())
    country = d[e.index(max(e))]#obtaining the country value


    return country

country_of_origin(list_of_doc[4])


'England'

In [398]:
def date_of_birth(doc):#To obtain the date of birth of the player
    #code goes here
    sentence = sent_tokenize(doc)[0]#tokenization of the sentence
    match = re.compile(r'born\b\s*((?:\S+\s+){0,3})')#using regex to extract the values
    date = match.findall(sentence)[0]
    date = re.sub('\W+',' ', date )
    
    return date

Execute the below command to check your fuction


In [399]:
date_of_birth(list_of_doc[2])

'5 February 1992 '

Expected output '5 February 1992'

In [400]:
def team_of_the_player(doc):#to obtain the team of the player
    
    sent = sent_tokenize(doc)
    tm = []
    a = []
    b = []

    tag = ie_preprocess(doc)
    for i, sent in enumerate(sent):#checking the initial lines
        if i == 2:
            break
        m = re.compile(r'((?:[\S,]+\s+){0,1})national team')#regex to get national team
        m_a = re.compile(r'a\s+((?:[\S,]+\s*){0,2})')#regex to get club

        team = m.findall(sent)#finding national team

        if len(m_a.findall(sent)) != 0:
            for f in m_a.findall(sent):
                b.append(f)#appending clubs based on key words

        a.append(named_entity_finding(tag[i], False, "ORGANIZATION"))
        if len(team) != 0:
            l = team[0]+"national team"
            tm.append(l)#finding national team

    if len(list(set(tm))) != 0:
        n = list(set(tm))[0]#obtaining the unique list of values
    else:
        n = country_of_origin(doc) +" national team"

    a = list(itertools.chain.from_iterable(a))   # Flattening the list

    nclub = []
    for q in b:
        nclub.extend(nltk.word_tokenize(q))

    for i, s in enumerate(b):
        b[i] = b[i].rstrip()
    if len(list(set(a).intersection(b))) != 0:
        a = list(set(a).intersection(b))
    a.append(n)#appending the values


    team = list(set(a).difference(nltk.word_tokenize(sent[0])[0:6]))#obtaining the team

    return team


team_of_the_player(list_of_doc[4])


['England national team']

In [401]:
def position_of_the_player(doc):#to obtain the position of the player
    pos = ["forward", "captian", "attacking midfielder", "striker", "winger", "central midfielder", "defensive tackle", "defensive end"]
    position = []#various positions in football is generated as a list
    sent = sent_tokenize(doc)#tokenization of the sentence
    for i, sent in enumerate(sent):
        for x in pos:
            rex = re.compile(r'\b({0})\b'.format(x), flags=re.IGNORECASE)#using regex to find matching values
            r = bool(rex.search(sent))
            if r == True:
                position.append(x)#appending to position when value is true

    return position

position_of_the_player(list_of_doc[4])

['forward']

# Task 5 (10 Marks)

Write a function using the outputs from the previous functions to generate JSON-LD output as follows.

Reference: https://json-ld.org/primer/latest/

{ "@id": "http://my-soccer-ontology.com/footballer/name_of_the_player",

    "name": "",
    "born": "",
    "country": "",
    "position": [
        { "@id": "http://my-soccer-ontology.com/position",
            "type": ""
        }
     ]   
     "team": [
        { "@id": "http://my-soccer-ontology.com/team",
            "name": ""
        }   
     ]
}


In [402]:
from pyld import jsonld
import json
def data_generator(doc):#generating the data
    
    data = [name_of_the_player(doc), date_of_birth(doc), country_of_origin(doc), position_of_the_player(doc), team_of_the_player(doc)]
    
    return data


data = data_generator(list_of_doc[4])#storing it here
print(data)

['Wayne Mark Rooney', '24 October 1985 ', 'England', ['forward'], ['England national team']]


In [403]:
def generate_jsonld(arg):#displaying the data based on json ld
    ld = { "@id": "http://my-soccer-ontology.com/footballer/ "+arg[0],

        "name": arg[0],
        "born": arg[1],
        "country": arg[2],
        "position": [
            { "@id": "http://my-soccer-ontology.com/position/",
                "type": arg[3]
            }
         ],   
         "team": [
            { "@id": "http://my-soccer-ontology.com/team/",
                "name": arg[4]
            }   
         ],
           
    }

    return json.dumps(ld,indent = 3, separators=(',', ': '))


data = data_generator(list_of_doc[4])
print(generate_jsonld(data))



{
   "@id": "http://my-soccer-ontology.com/footballer/ Wayne Mark Rooney",
   "name": "Wayne Mark Rooney",
   "born": "24 October 1985 ",
   "country": "England",
   "position": [
      {
         "@id": "http://my-soccer-ontology.com/position/",
         "type": [
            "forward"
         ]
      }
   ],
   "team": [
      {
         "@id": "http://my-soccer-ontology.com/team/",
         "name": [
            "England national team"
         ]
      }
   ]
}


# Task 6 (10 Marks)
Identify one other relation (besides team and player) and write a function to extract this. Also extend the JSON-LD output accordingly.

In [404]:
def debutyear(doc):#to obtain the debut year of the player
    sent = sent_tokenize(doc)
    deb = []
    ag = []
    for se in sent:
        sp_sent = se.split()
        if "debut" in sp_sent:
            date = re.findall('\d{4}', " ".join(sp_sent))#when debut year is present finding it using regex
            if len(date) !=0:
                deb.append(date[0])#appending the value of the year
    if len(deb) == 0:#when debut year is not given
        deb.append("Not Available")
   
    return [deb[0]]
debutyear(list_of_doc[4])

['2002']

In [405]:
def data_generator(doc):#generating the data along with debut year
    data = [name_of_the_player(doc), date_of_birth(doc), country_of_origin(doc), position_of_the_player(doc), team_of_the_player(doc), debutyear(doc)]
    return data
data = data_generator(list_of_doc[4])
print(data)

['Wayne Mark Rooney', '24 October 1985 ', 'England', ['forward'], ['England national team'], ['2002']]


In [407]:
def generate_jsonld(arg):#adding debut year to the json ld


        ld = { "@id": "http://my-soccer-ontology.com/footballer/ "+arg[0],

            "name": arg[0],
            "born": arg[1],
            "country": arg[2],
            "position": [
                { "@id": "http://my-soccer-ontology.com/position/",
                    "type": arg[3]
                }
             ],   
             "team": [
                { "@id": "http://my-soccer-ontology.com/team/",
                    "name": arg[4]
                }   
             ],
               "Debut Year": arg[5]
        }

        return json.dumps(ld,indent = 3, separators=(',', ': '))
    

data = data_generator(list_of_doc[4])
print(generate_jsonld(data))


{
   "@id": "http://my-soccer-ontology.com/footballer/ Wayne Mark Rooney",
   "name": "Wayne Mark Rooney",
   "born": "24 October 1985 ",
   "country": "England",
   "position": [
      {
         "@id": "http://my-soccer-ontology.com/position/",
         "type": [
            "forward"
         ]
      }
   ],
   "team": [
      {
         "@id": "http://my-soccer-ontology.com/team/",
         "name": [
            "England national team"
         ]
      }
   ],
   "Debut Year": [
      "2002"
   ]
}
